# Input Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# CSV dataset file name
file_name = 'D1.csv'

In [ ]:
# Please put csv file in the same folder with this jupyter notebook
df = pd.read_csv(file_name)

In [ ]:
print(df.info())